# Undoing Changes in Git

**Don't panic!** Almost nothing in Git is truly permanent. Git has multiple safety nets, and the reflog keeps track of everything for 30+ days.

This notebook teaches you how to undo different types of mistakes in Git.

## Setup: Create a Test Repository

Let's create a safe playground to practice undoing changes.

In [ ]:
import os
import tempfile
import shutil

# Create a temporary directory for our test repo
test_dir = tempfile.mkdtemp(prefix="git_undo_practice_")
print(f"Created test repo at: {test_dir}")

# Change to that directory
os.chdir(test_dir)
print(f"Working in: {os.getcwd()}")

In [ ]:
# Initialize a Git repository
!git init
!git config user.name "Test User"
!git config user.email "test@example.com"

In [ ]:
# Create several commits to practice with
!echo "First line" > file1.txt
!git add file1.txt
!git commit -m "Commit 1: Add file1.txt"

!echo "Second line" >> file1.txt
!echo "New file" > file2.txt
!git add .
!git commit -m "Commit 2: Update file1 and add file2"

!echo "Third line" >> file1.txt
!git add file1.txt
!git commit -m "Commit 3: Add third line"

print("\n=== Commit History ===")
!git log --oneline

## 1. Discarding Working Directory Changes

**Scenario:** You modified a file but want to throw away those changes.

**Two commands do this:**
- Old way: `git checkout -- <file>`
- New way: `git restore <file>` (Git 2.23+, clearer name)

In [ ]:
# Make a change we'll want to undo
!echo "Oops, this is a mistake!" >> file1.txt

print("=== File content after adding mistake ===")
!cat file1.txt

print("\n=== Git status ===")
!git status

In [ ]:
# Restore the file to the last committed version
!git restore file1.txt

print("=== File content after restore ===")
!cat file1.txt

print("\n=== Git status ===")
!git status

### What happened?

1. We added "Oops, this is a mistake!" to `file1.txt`
2. `git restore file1.txt` threw away the working directory changes
3. The file went back to how it looked in the last commit

**WARNING:** This is destructive! The discarded changes are gone forever (they were never committed).

## 2. Unstaging Files

**Scenario:** You ran `git add` but want to un-add the file (keep the changes, just remove from staging area).

**Two commands do this:**
- Old way: `git reset HEAD <file>`
- New way: `git restore --staged <file>` (clearer!)

In [ ]:
# Make a change and stage it
!echo "New feature" >> file2.txt
!git add file2.txt

print("=== Git status after staging ===")
!git status

In [ ]:
# Unstage the file (but keep the changes in working directory)
!git restore --staged file2.txt

print("=== Git status after unstaging ===")
!git status

print("\n=== File still has the changes ===")
!cat file2.txt

### What happened?

1. We added a line to `file2.txt` and staged it with `git add`
2. `git restore --staged file2.txt` removed it from the staging area
3. The changes are still in the file (in your working directory)
4. The file is now "modified but not staged"

**This is safe!** You didn't lose any work.

## 3. Git Reset: Undoing Commits

`git reset` moves the branch pointer backward. It has **three modes** that affect different areas:

```
Git has three "areas" where your code lives:

[Working Directory]  <- What you see in your files
         ↓
[Staging Area]       <- What's queued for next commit (via git add)
         ↓
[Repository/HEAD]    <- Committed snapshots


git reset --soft HEAD~1:
    [Working Directory]  ← Unchanged
    [Staging Area]       ← Unchanged  
    [HEAD]               ← Moved back 1 commit ✓
    
    Use case: "I want to recommit with a better message"


git reset --mixed HEAD~1:  (this is the DEFAULT)
    [Working Directory]  ← Unchanged
    [Staging Area]       ← Reset ✓
    [HEAD]               ← Moved back 1 commit ✓
    
    Use case: "I want to uncommit and re-add files differently"


git reset --hard HEAD~1:
    [Working Directory]  ← Reset ✓
    [Staging Area]       ← Reset ✓
    [HEAD]               ← Moved back 1 commit ✓
    
    Use case: "Throw everything away, go back in time"
    ⚠️  DANGEROUS: Discards uncommitted changes!
```

### 3a. Reset --soft (Undo commit, keep changes staged)

In [ ]:
# First, let's see our current state
print("=== Current commit history ===")
!git log --oneline

print("\n=== Current file1.txt content ===")
!cat file1.txt

In [ ]:
# Undo the last commit with --soft
!git reset --soft HEAD~1

print("=== After reset --soft ===")
!git log --oneline

print("\n=== Git status ===")
!git status

print("\n=== File content (unchanged) ===")
!cat file1.txt

### What happened with --soft?

1. "Commit 3: Add third line" disappeared from history
2. The third line is **still in the file** (working directory unchanged)
3. The change is **already staged** (ready to commit again)
4. You could now edit the commit message and recommit

**Use --soft when:** You want to fix a commit message or combine commits.

In [ ]:
# Let's recommit with a better message
!git commit -m "Commit 3 (improved): Add third line with better explanation"

print("=== New commit history ===")
!git log --oneline

### 3b. Reset --mixed (Undo commit, unstage changes)

In [ ]:
# Undo the last commit with --mixed (this is the default if you don't specify)
!git reset --mixed HEAD~1

print("=== After reset --mixed ===")
!git log --oneline

print("\n=== Git status ===")
!git status

print("\n=== File content (still unchanged) ===")
!cat file1.txt

### What happened with --mixed?

1. Commit was undone (same as --soft)
2. The third line is **still in the file** (working directory unchanged)
3. The change is **NOT staged** (you'd need to `git add` again)

**Use --mixed when:** You want to uncommit and re-organize which files go in which commits.

In [ ]:
# Clean up: Re-add and commit
!git add file1.txt
!git commit -m "Commit 3: Add third line"

print("=== Back to original state ===")
!git log --oneline

### 3c. Reset --hard (Undo commit AND discard changes)

In [ ]:
# Make a commit we'll completely undo
!echo "Temporary change" >> file1.txt
!git add file1.txt
!git commit -m "Commit 4: Temporary commit to undo"

print("=== Before reset --hard ===")
!git log --oneline
print("\nFile content:")
!cat file1.txt

In [ ]:
# DANGER ZONE: This will discard changes!
!git reset --hard HEAD~1

print("=== After reset --hard ===")
!git log --oneline

print("\n=== Git status (clean) ===")
!git status

print("\n=== File content (fourth line gone!) ===")
!cat file1.txt

### What happened with --hard?

1. Commit 4 disappeared
2. "Temporary change" was **deleted from the file**
3. Working directory is clean (matches the commit we reset to)

**Use --hard when:** You want to completely throw away recent work.

**WARNING:** --hard is destructive! But don't panic - the reflog can still save you (we'll see that later).

## 4. Git Revert: The "Undo" Commit

`git revert` creates a **new commit** that undoes changes from a previous commit.

**Key difference from reset:**
- `reset` = rewrites history (moves branch pointer backward)
- `revert` = adds to history (creates new commit)

```
Before revert:
A -- B -- C -- D    <- main

After git revert C:
A -- B -- C -- D -- C'   <- main
                    ^
                    New commit that undoes C
```

### When to use revert vs reset:

**Use RESET for:**
- Local commits you haven't pushed yet
- When you're working alone
- When you want to "erase" history

**Use REVERT for:**
- Commits that have been pushed to a shared repository
- When others have already pulled your changes
- When you want to keep history intact (auditable)

**Rule of thumb:** If you've pushed it, use revert. If it's still local, reset is fine.

In [ ]:
# Let's revert commit 2 (which added file2.txt)
print("=== Current commits ===")
!git log --oneline

print("\n=== Files before revert ===")
!ls -la *.txt

In [ ]:
# Get the commit hash of "Commit 2" (second from bottom)
import subprocess
result = subprocess.run(["git", "log", "--oneline"], capture_output=True, text=True)
commits = result.stdout.strip().split('\n')
commit2_hash = commits[-2].split()[0]  # Second commit from bottom
print(f"Reverting commit: {commits[-2]}")

# Revert that commit
!git revert {commit2_hash} --no-edit

print("\n=== After revert ===")
!git log --oneline

In [ ]:
print("=== Files after revert ===")
!ls -la *.txt 2>&1 || echo "(file2.txt is gone!)"

print("\n=== file1.txt content ===")
!cat file1.txt

### What happened?

1. Commit 2 originally added `file2.txt` and modified `file1.txt`
2. `git revert` created a **new commit** that does the opposite
3. `file2.txt` was deleted (undoing the add)
4. `file1.txt` went back to its state before commit 2
5. All commits are still in the history (nothing was erased)

**This is safe for shared repositories!** Other developers can pull this change without conflicts.

## 5. Git Stash: Temporarily Shelving Work

**Analogy:** Stash is like putting your work in a drawer temporarily. You can take it back out later.

**Use stash when:**
- You need to switch branches but have uncommitted changes
- You want to try something experimental without committing
- Someone asks you to fix a bug urgently (stash current work, fix bug, pop stash back)

In [ ]:
# Make some changes we'll stash
!echo "Work in progress" >> file1.txt
!echo "Another file" > file3.txt

print("=== Changes before stash ===")
!git status

In [ ]:
# Stash the changes
!git stash push -m "WIP: Feature in progress"

print("=== After stash ===")
!git status

print("\n=== file1.txt (changes are gone) ===")
!cat file1.txt

### What happened?

1. We had uncommitted changes in `file1.txt` and a new `file3.txt`
2. `git stash` saved them to a "stash stack" and cleaned the working directory
3. Now we have a clean state (as if we never made those changes)
4. The changes aren't lost - they're in the stash!

In [ ]:
# View what's in the stash
!git stash list

In [ ]:
# Bring the changes back
!git stash pop

print("=== After stash pop ===")
!git status

print("\n=== file1.txt (changes are back!) ===")
!cat file1.txt

### What happened with pop?

1. `git stash pop` took the most recent stash
2. Applied those changes back to the working directory
3. Removed that stash from the stack ("pop" like a stack data structure)

### Other useful stash commands:

- `git stash apply` - Apply stash but keep it in the list (don't pop it off)
- `git stash drop` - Delete a stash without applying it
- `git stash clear` - Delete all stashes
- `git stash show` - See what's in a stash

## 6. The Reflog: Your Safety Net

The **reflog** is Git's diary. It records every time HEAD moves, even if you deleted commits with `reset --hard`.

**Reflog keeps deleted commits for ~30 days!**

Think of it as an "undo" for Git commands themselves.

In [ ]:
# Let's create a commit and then "lose" it with reset --hard
!echo "Important work" > important.txt
!git add important.txt
!git commit -m "Important commit we'll accidentally delete"

print("=== Created important commit ===")
!git log --oneline | head -n 3

In [ ]:
# Save the commit hash before we delete it
import subprocess
result = subprocess.run(["git", "rev-parse", "HEAD"], capture_output=True, text=True)
important_commit = result.stdout.strip()
print(f"Important commit hash: {important_commit}")

# Now delete it with reset --hard
!git reset --hard HEAD~1

print("\n=== After reset --hard (commit looks gone!) ===")
!git log --oneline | head -n 3

print("\n=== important.txt is gone too ===")
!ls important.txt 2>&1 || echo "File not found"

In [ ]:
# But the reflog remembers!
print("=== Reflog shows everything ===")
!git reflog | head -n 10

### Reading the reflog:

Each line shows:
- **Commit hash** - Where HEAD pointed
- **HEAD@{N}** - How many moves ago (HEAD@{0} is current, HEAD@{1} is previous, etc.)
- **Action** - What command moved HEAD
- **Message** - Description of the action

You can see:
- The `reset: moving to HEAD~1` that "deleted" our commit
- The commit we made before that
- All previous actions

In [ ]:
# Recover the "lost" commit
!git reset --hard {important_commit}

print("=== Recovered! ===")
!git log --oneline | head -n 3

print("\n=== File is back ===")
!cat important.txt

### What just happened?

1. We "deleted" a commit with `reset --hard`
2. The commit was still in the reflog (Git keeps everything for ~30 days)
3. We used the commit hash from the reflog to reset back to it
4. Everything was recovered!

**Key lesson:** Don't panic if you "lose" a commit. Check the reflog first!

## 7. Decision Guide: Which Command to Use?

Here's a quick guide for which undo command to use:

### Scenario 1: Modified a file, want to discard changes
**Command:** `git restore <file>` (or `git checkout -- <file>`)
- Changes were never committed
- ⚠️ Destructive: changes are lost forever

### Scenario 2: Staged a file (git add), want to unstage
**Command:** `git restore --staged <file>` (or `git reset HEAD <file>`)
- File changes are kept
- Only removes from staging area
- Safe: no data loss

### Scenario 3: Committed locally, want to undo (NOT pushed yet)
**Command:** `git reset`
- `--soft` = Keep changes staged (recommit with new message)
- `--mixed` = Keep changes unstaged (re-organize commits)
- `--hard` = Throw away changes completely
- Rewrites history (don't use on pushed commits!)

### Scenario 4: Committed AND pushed, want to undo
**Command:** `git revert <commit>`
- Creates a new commit that undoes the changes
- Doesn't rewrite history
- Safe for shared repositories

### Scenario 5: Need to switch tasks temporarily
**Command:** `git stash`
- Saves uncommitted changes
- Use `git stash pop` to restore them later
- Like putting work in a drawer

### Scenario 6: Accidentally deleted commits
**Command:** `git reflog` + `git reset --hard <hash>`
- Find the lost commit in reflog
- Reset to it
- Your safety net!

## 8. Quick Reference: Command Cheat Sheet

```bash
# Discard working directory changes
git restore <file>              # Modern way
git checkout -- <file>          # Old way

# Unstage files
git restore --staged <file>     # Modern way  
git reset HEAD <file>           # Old way

# Undo commits (local only!)
git reset --soft HEAD~1         # Undo commit, keep changes staged
git reset --mixed HEAD~1        # Undo commit, keep changes unstaged (default)
git reset --hard HEAD~1         # Undo commit, delete changes

# Undo commits (safe for pushed)
git revert <commit-hash>        # Create undo commit

# Stash (temporary shelving)
git stash                       # Save changes
git stash push -m "message"     # Save with message
git stash list                  # View stashes
git stash pop                   # Restore and remove from stack
git stash apply                 # Restore but keep in stack
git stash drop                  # Delete a stash

# Safety net
git reflog                      # View all HEAD movements
git reset --hard <commit>       # Go to any commit in reflog
```

## 9. Practice Exercise

Try these scenarios to test your understanding:

### Exercise 1: Unstaging
1. Create a file `practice1.txt` with some text
2. Stage it with `git add`
3. Unstage it using the modern command
4. Verify the file still has your changes

In [ ]:
# Your turn: Exercise 1
# Write your commands here


### Exercise 2: Reset modes
1. Create a file `practice2.txt` and commit it
2. Add a line and commit again
3. Use `git reset --mixed HEAD~1` to undo the last commit
4. Verify the line is still in your file but not staged

In [ ]:
# Your turn: Exercise 2
# Write your commands here


### Exercise 3: Stash workflow
1. Make changes to `file1.txt` (don't commit)
2. Stash the changes with a descriptive message
3. Verify the working directory is clean
4. Pop the stash to restore your changes

In [ ]:
# Your turn: Exercise 3
# Write your commands here


### Exercise 4: Reflog recovery
1. Create a commit with a file `precious.txt`
2. Use `git reset --hard HEAD~1` to delete it
3. Use `git reflog` to find the deleted commit
4. Recover it using `git reset --hard <hash>`

In [ ]:
# Your turn: Exercise 4
# Write your commands here


## 10. Solutions

Try the exercises yourself first! Solutions below.

### Solution 1: Unstaging

In [ ]:
# Solution 1
!echo "Practice text" > practice1.txt
!git add practice1.txt

print("=== After staging ===")
!git status

# Unstage using modern command
!git restore --staged practice1.txt

print("\n=== After unstaging ===")
!git status

print("\n=== File still has content ===")
!cat practice1.txt

### Solution 2: Reset modes

In [ ]:
# Solution 2
!echo "Initial content" > practice2.txt
!git add practice2.txt
!git commit -m "Add practice2.txt"

!echo "Second line" >> practice2.txt
!git add practice2.txt  
!git commit -m "Add second line"

print("=== Before reset ===")
!git log --oneline | head -n 3

# Undo last commit with --mixed
!git reset --mixed HEAD~1

print("\n=== After reset --mixed ===")
!git log --oneline | head -n 3

print("\n=== Status (unstaged changes) ===")
!git status

print("\n=== File still has both lines ===")
!cat practice2.txt

### Solution 3: Stash workflow

In [ ]:
# Solution 3
!echo "Temporary work" >> file1.txt

print("=== Before stash ===")
!git status

# Stash with message
!git stash push -m "Temporary changes to file1"

print("\n=== After stash (clean) ===")
!git status

print("\n=== Stash list ===")
!git stash list

# Pop it back
!git stash pop

print("\n=== After pop (changes restored) ===")
!git status

### Solution 4: Reflog recovery

In [ ]:
# Solution 4
!echo "Precious data" > precious.txt
!git add precious.txt
!git commit -m "Add precious file"

# Save the commit hash
import subprocess
result = subprocess.run(["git", "rev-parse", "HEAD"], capture_output=True, text=True)
precious_commit = result.stdout.strip()
print(f"Created commit: {precious_commit}")

# Delete it
!git reset --hard HEAD~1

print("\n=== After deletion ===")
!ls precious.txt 2>&1 || echo "precious.txt is gone!"

print("\n=== Reflog shows the deleted commit ===")
!git reflog | head -n 5

# Recover it
!git reset --hard {precious_commit}

print("\n=== Recovered! ===")
!cat precious.txt

## Key Takeaways

1. **Don't panic!** Git's reflog keeps "deleted" commits for ~30 days

2. **Restoring files:**
   - `git restore <file>` - Discard working directory changes
   - `git restore --staged <file>` - Unstage files

3. **Reset modes (for local commits):**
   - `--soft` = Move HEAD, keep staging and working directory
   - `--mixed` = Move HEAD, reset staging, keep working directory (default)
   - `--hard` = Move HEAD, reset everything (dangerous!)

4. **Revert vs Reset:**
   - `reset` = Rewrites history (use for unpushed commits)
   - `revert` = Creates new commit (safe for pushed commits)

5. **Stash = Temporary drawer:**
   - Save work without committing
   - Perfect for switching contexts
   - `git stash` → do other work → `git stash pop`

6. **Reflog = Your safety net:**
   - Records all HEAD movements
   - Can recover "lost" commits
   - Check it before panicking!

7. **Rule of thumb:**
   - Pushed commits → use `revert`
   - Local commits → `reset` is fine
   - When in doubt → `stash` or make a backup branch

## Cleanup

Run this cell to delete the test repository:

In [ ]:
# Change back to original directory and clean up
import os
os.chdir("..")
if os.path.exists(test_dir):
    shutil.rmtree(test_dir)
    print(f"Cleaned up test repository at {test_dir}")
else:
    print("Test directory already cleaned up")